In [9]:
import wandb
import pandas as pd
import datetime
import tqdm

username = "memmelma"
project = "final"

filter_name = "vit_b"
filter_type = "eval"
filter_state = "finished"

metrics = ["eval_metricsspl", "eval_metricsdistance_to_goal", "eval_metricssoftspl", "eval_metricssuccess"]

configs = []

out_name = "experiments_2020_vit_b"

In [10]:
# retreive runs

run_ids = []
run_names = []
run_ckpts = []

api = wandb.Api(timeout=45)
runs = api.runs(f"{username}/{project}")

runs_df = pd.DataFrame(columns=["name", "id", "state", *configs, *metrics])

for run in tqdm.tqdm(runs):
    
    if filter_state and run._state != filter_state:
        continue
    
    if filter_type and filter_type not in run.name:
        continue
        
    if filter_name and filter_name not in run.name:
        continue
    
    run_dict = {}
    
    # add base attributes
    run_dict["name"] = run.name.split("[")[0]
    run_dict["id"] = run.id 
    run_dict["state"] = run._state
    
    # add config
    for config in configs:
        # traverse config keys
        run_config_tmp = run.config
        for config_iter in config.split('.'):
            run_config_tmp = run_config_tmp[config_iter]
        run_dict[config] = run_config_tmp
    
    # add filter_operation filter_metric
    run_history = run.history()
    
    # add other metrics corresponding to filter_operation filter_metric
    for metric in metrics:
        # catch models that only evaluate some actions
        if metric in run_history.keys():
            run_dict[metric] = run_history[metric][0]
        else:
            run_dict[metric] = -1
    
    runs_df = runs_df.append(run_dict, ignore_index=True)
    
runs_df

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 82/82 [00:03<00:00, 25.88it/s]


,name,id,state,eval_metricsspl,eval_metricsdistance_to_goal,eval_metricssoftspl,eval_metricssuccess
0,eval_vit_b_mmae_rgbd,5q46ubjm,finished,0.650517,0.513097,0.695450,0.851107
1,eval_vit_b_mmae_act_rgbd_freeze,ovik9t52,finished,0.094237,2.495161,0.458835,0.127767
2,eval_vit_b_in21k_act_rgb_freeze,3qapfo08,finished,0.093541,2.759619,0.454583,0.126761
3,eval_vit_b_in21k_act_rgb,34ne5jbv,finished,0.460963,0.681423,0.672055,0.602616
4,eval_vit_b_dino_act_rgb,1yiin9jx,finished,0.439565,0.783047,0.663046,0.578471
5,eval_vit_b_mmae_act_rgbd,246dr45g,finished,0.679063,0.421332,0.713014,0.882294
6,eval_vit_b_in21k_act_rgbd_freeze,2jv9d4f5,finished,0.097675,2.370498,0.475481,0.130785
7,eval_vit_b_dino_act_rgbd,s2qbq0md,finished,0.654726,0.439060,0.711645,0.850101
8,eval_vit_b_in21k_act_rgbd,3mc2xl7s,finished,0.699632,0.381328,0.715061,0.914487
9,eval_vit_b_dino_act_rgb_freeze,ldy30iih,finished,0.084547,2.408709,0.464224,0.112676
